In [302]:
import math

import numpy as np 

from nasbench import api

import utils

In [8]:
nasbench = api.NASBench('../nasbench_full.tfrecord')

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 182 seconds


In [173]:
model_counts = 10000

hashes = nasbench.fixed_statistics.keys()
N = len(hashes)

sampled_hashes = random.sample(hashes, model_counts) # list
edit_util = utils.EditUtils(nasbench)

In [314]:
stats = dict()
edit_count = 1
for module_hash in sampled_hashes[:1]:
    fixed_stats, computed_stats = nasbench.get_metrics_from_hash(module_hash)

    adj_matrix = fixed_stats['module_adjacency']
    ops = fixed_stats['module_operations']
    
    repeats = [repeat['final_test_accuracy'] for repeat in computed_stats[108]]
    model_accuracy = sum(repeats) / len(repeats)

    model_counts_per_original = 5
    generated_matrix = set()
    generated_ops = set()
    accuracies = []
    stat = dict()
    for iteration in range(model_counts_per_original * 10):
        print("iteration: {}".format(iteration))
        new_matrix, new_ops = edit_util.edit_model(adj_matrix, ops, edit_count)

        new_model = api.ModelSpec(
           matrix=new_matrix,
           ops=new_ops,
        )
        
        matrix_tuple = tuple(new_matrix.reshape(1, -1)[0])
        ops_tuple = tuple(ops)
        if matrix_tuple in generated_matrix and ops_tuple in generated_ops: 
            continue
        else:
            generated_matrix.add(matrix_tuple)
            generated_ops.add(ops_tuple)

        _, new_computed_stats = nasbench.get_metrics_from_spec(new_model)
        repeats = [repeat['final_test_accuracy'] for repeat in new_computed_stats[108]]
        new_model_accuracy = sum(repeats) / len(repeats)
        accuracies.append(new_model_accuracy)
        
        if len(accuracies) > model_counts_per_original: break


    diffs = [abs(acc - model_accuracy) for acc in accuracies]
    avg = sum(diffs)/len(diffs)
    median = diffs[len(diffs)//2+1]
    stdev = math.sqrt(sum(map(lambda x : (x - avg)**2, diffs))/len(diffs))
    
    stat['adj_matrix'] = adj_matrix
    stat['operations'] = ops
    stat['model_accuracy'] = model_accuracy
    stat['accuracies'] = accuracies
    stat['diffs'] = diffs
    stat['avg'] = avg
    stat['standard_deviation'] = stdev
    stat['median'] = median
    
    stats[module_hash] = stat
stats

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
iteration: 31
iteration: 32
iteration: 33
iteration: 34
iteration: 35
iteration: 36
iteration: 37
iteration: 38
iteration: 39
iteration: 40
iteration: 41
iteration: 42
iteration: 43
iteration: 44
iteration: 45
iteration: 46
iteration: 47
iteration: 48
iteration: 49


{'acf9e51be18ca48cf8e81b900a9f81c8': {'adj_matrix': array([[0, 1, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0]], dtype=int8),
  'operations': ['input',
   'conv1x1-bn-relu',
   'conv1x1-bn-relu',
   'conv1x1-bn-relu',
   'conv3x3-bn-relu',
   'conv1x1-bn-relu',
   'output'],
  'model_accuracy': 0.9054487347602844,
  'accuracies': [0.9054487347602844, 0.9091546535491943, 0.8645165761311849],
  'diffs': [0.0, 0.003705918788909912, 0.04093215862909949],
  'avg': 0.014879359139336468,
  'standard_deviation': 0.01848413246468163,
  'median': 0.04093215862909949}}

In [264]:
from imp import reload
reload(utils)
edit_util = utils.EditUtils(nasbench)

In [239]:
edit_util.edit_edge(adj_matrix, edit_count)

inside forloop
not have
{(1, 2), (2, 6), (4, 6), (1, 4), (0, 6), (1, 5), (2, 5), (0, 5), (1, 6), (0, 4), (0, 3), (2, 4)}
{0, 3}
{3, 5, 6}


array([[0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [275]:
edit_count=2
edit_util.edit_model(adj_matrix, ops, edit_count)

(array([[0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8),
 ['input',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'conv1x1-bn-relu',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'output'])

In [273]:
ops, adj_matrix

(['input',
  'conv1x1-bn-relu',
  'conv1x1-bn-relu',
  'conv1x1-bn-relu',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'output'],
 array([[0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8))